In [1]:
import pandas as pd
import numpy as np
import os    
from chardet import detect
import string
import re
import nltk
nltk.download('punkt')
from pandas.api.types import is_string_dtype
import ast
import pattern
from spellchecker import SpellChecker
import time
import gensim.models.word2vec as w2v

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\latee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Write a cleaning script that will load the raw data and whip it into shape

#### To complete this challenge, determine the 

1. the five most common journals and the total articles for each
1. calculate the mean, median, and standard deviation of the open-access cost per article for each journal

In [2]:
PATH = r'C:\Users\latee\Downloads\WELLCOME_APCspend2013_forThinkful.csv'

In [3]:
# I detected a encoding error when attempting to read the file as a csv
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa3 in position 6: invalid start byte
# The following returns who encoded the data
def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read()
    return detect(rawdata)['encoding']
get_encoding_type(PATH)

'Windows-1254'

##### Switching the engine sovled the problem by allowing us to be more inclusive in the types of characters we can import

In [4]:
APC_SPEND = pd.read_csv(PATH, engine='python')
orginal = APC_SPEND.copy()

In [5]:
# Making Columns Lowercase
APC_SPEND.columns = map(lambda x:x.lower(), APC_SPEND.columns)
# Recasting the Column as (string type) & removing digits new char lines and other special chars 
APC_SPEND['journal title'] = APC_SPEND['journal title'].str.lower().astype(str)
APC_SPEND['journal title'] = APC_SPEND['journal title'].map(lambda x: re.sub(r'[\d\n,.&:-]', '', x))
# Renaming the Cost Column & Removing Special Chars 
APC_SPEND['cost'] = APC_SPEND["cost (£) charged to wellcome (inc vat when charged)"]
APC_SPEND = APC_SPEND.drop(['cost (£) charged to wellcome (inc vat when charged)'], axis=1)
APC_SPEND['cost'] = APC_SPEND['cost'].map(lambda x: re.sub(r'[^\d\n,.]', '', x)).astype('float32')

In [6]:
APC_SPEND

,pmid/pmcid,publisher,journal title,article title,cost
0,NaN,CUP,psychological medicine,Reduced parahippocampal cortical thickness in ...,0.000000
1,PMC3679557,ACS,biomacromolecules,Structural characterization of a Model Gram-ne...,2381.040039
2,23043264 PMC3506128,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.559998
3,23438330 PMC3646402,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,669.640015
4,23438216 PMC3601604,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,685.880005
...,...,...,...,...,...
2122,2901593,Wolters Kluwer Health,circulation research,Mechanistic Links Between Na+ Channel (SCN5A) ...,1334.150024
2123,3748854,Wolters Kluwer Health,aids,Evaluation of an empiric risk screening score ...,1834.770020
2124,3785148,Wolters Kluwer Health,pediatr infect dis j,Topical umbilical cord care for prevention of ...,1834.770020
2125,PMCID:\n PMC3647051\n,Wolters Kluwer N.V./Lippinott,aids,Grassroots Community Organisations' Contributi...,2374.520020


In [7]:
def null_values(df):
    for cols in df.columns:
        print('Null Values in ', cols, df[cols].isnull().sum())
    return
null_values(APC_SPEND)

Null Values in  pmid/pmcid 199
Null Values in  publisher 0
Null Values in  journal title 0
Null Values in  article title 0
Null Values in  cost 0


In [8]:
spell = SpellChecker()
stop_words = nltk.corpus.stopwords.words('english')

In [9]:
t0 = time.time()

# Removing Stop Words & Spell Checking the Journal Title Column
APC_SPEND['journal title'] = APC_SPEND['journal title'].apply(lambda x: [spell.correction(j_names) for j_names in x.split()]).astype(str)

t1 = time.time()
total = t1-t0
print(total)

406.54006934165955


In [10]:
APC_SPEND['journal title']

0                           ['psychological', 'medicine']
1                                   ['biomacromolecules']
2                                    ['j', 'med', 'chem']
3                                    ['j', 'med', 'chem']
4                                    ['j', 'org', 'chem']
                              ...                        
2122                          ['circulation', 'research']
2123                                             ['aids']
2124                    ['mediate', 'infect', 'dis', 'j']
2125                                             ['aids']
2126    ['journal', 'of', 'acquired', 'immune', 'defic...
Name: journal title, Length: 2127, dtype: object

#### The Top 5 Journals Are:

- Plos One
- Journal of Biological Chemistry
- Neuroimage
- Nucleic Acids Research
- Plos Genetics

#### The Mean, Median, and Standard Deviation of each of the Journals is Calulated below

Keep in mind that I found 800 unique journals so you could always print out the entire list using:

with pd.option_context('display.max_rows'): 

    print([avg | median | std])

In [11]:
avg = APC_SPEND.groupby('journal title').cost.agg(np.mean)

In [12]:
median = APC_SPEND.groupby('journal title').cost.agg(np.median)

In [13]:
std = APC_SPEND.groupby('journal title').cost.agg(np.std)

In [14]:
avg

journal title
['academy', 'of', 'nutrition', 'and', 'dietetics']                                 2379.540039
['acs', 'chemical', 'biology']                                                     1418.186035
['acs', 'chemical', 'neuroscience']                                                1186.800049
['acs', 'nano']                                                                     668.140015
['acta', 'crystallographic', 'section', 'd', 'biological', 'crystallography']       772.579956
                                                                                     ...      
['virology']                                                                       1928.505005
['virus', 'research']                                                              1947.089966
['vision', 'research']                                                           500727.593750
['visual', 'neuroscience']                                                         2034.000000
['zoonoses', 'and', 'public', 'healt

In [15]:
median

journal title
['academy', 'of', 'nutrition', 'and', 'dietetics']                                 2379.540039
['acs', 'chemical', 'biology']                                                     1294.589966
['acs', 'chemical', 'neuroscience']                                                1186.800049
['acs', 'nano']                                                                     668.140015
['acta', 'crystallographic', 'section', 'd', 'biological', 'crystallography']       772.579956
                                                                                     ...      
['virology']                                                                       1928.505005
['virus', 'research']                                                              1947.089966
['vision', 'research']                                                           500727.593750
['visual', 'neuroscience']                                                         2034.000000
['zoonoses', 'and', 'public', 'healt

In [16]:
std

journal title
['academy', 'of', 'nutrition', 'and', 'dietetics']                                         NaN
['acs', 'chemical', 'biology']                                                      507.309540
['acs', 'chemical', 'neuroscience']                                                        NaN
['acs', 'nano']                                                                      35.708893
['acta', 'crystallographic', 'section', 'd', 'biological', 'crystallography']         1.640493
                                                                                     ...      
['virology']                                                                        697.850708
['virus', 'research']                                                                      NaN
['vision', 'research']                                                           706076.375000
['visual', 'neuroscience']                                                                 NaN
['zoonoses', 'and', 'public', 'healt